In [1]:
import pandas as pd
import yfinance as yf
import sys
import os

sys.path.append(os.path.abspath("../Scripts"))
sys.path.append(os.path.abspath("../src"))

from Download_data import StockData

## Task 1: Preprocess and Explore the Data
### Load and Clean Data

In [2]:
tickers = ['TSLA', 'BND', 'SPY']
start_date = "2015-07-01"
end_date = "2025-07-31"

# Create instance of StockData
stock_data = StockData(tickers, start_date, end_date)


In [3]:
df = stock_data.download_data()

c:\Users\Specter\Documents\Tenx_Academy\Week-11\Time-Series-Forecasting-Portfolio-Optimization\src\Download_data.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  self.data = yf.download(self.tickers, start=self.start_date, end=self.end_date)
[*********************100%***********************]  3 of 3 completed


In [4]:
df.head()

Price           Close                              High              \
Ticker            BND         SPY       TSLA        BND         SPY   
Date                                                                  
2015-07-01  60.816715  174.917145  17.943333  60.914441  175.363920   
2015-07-02  60.967060  174.756943  18.667999  61.027202  175.566203   
2015-07-06  61.177608  174.259598  18.648001  61.222718  175.043557   
2015-07-07  61.237709  175.355453  17.858667  61.470750  175.481894   
2015-07-08  61.410625  172.413483  16.997334  61.463248  174.293311   

Price                        Low                              Open  \
Ticker           TSLA        BND         SPY       TSLA        BND   
Date                                                                 
2015-07-01  18.174667  60.764093  174.124747  17.856667  60.794164   
2015-07-02  18.830000  60.936989  174.335456  18.220667  60.936989   
2015-07-06  18.779333  61.057331  173.256457  18.420000  61.222718   
2015-07-07  18.346666  61.237709  172.059422  17.384666  61.410615   
2015-07-08  17.386667  61.312894  172.177451  16.954000  61.373036   

Price                               Volume                        
Ticker             SPY       TSLA      BND        SPY       TSLA  
Date                                                              
2015-07-01  175.111025  18.073999  5399300  135979900   31518000  
2015-07-02  175.397611  18.680000  1060100  104373700  107458500  
2015-07-06  173.458775  18.591999  2210700  117975400   61828500  
2015-07-07  174.461903  18.333332  4183200  173820200   91576500  
2015-07-08  174.006703  17.288000  1967800  164020100   93316500

In [5]:
# Extract different price categories into separate DataFrames.
close_df, high_df, low_df, open_df, volume_df = stock_data.extract_price_categories()

In [6]:
# Reset index of DataFrames.
close_df, high_df, low_df, open_df, volume_df = stock_data.reset_indexes()

In [7]:
close_df, high_df, low_df, open_df, volume_df = stock_data.melt_data()

In [8]:
# Get the merged DataFrame
stock_data.merge_data()
df_merged = stock_data.get_merged_data()

In [9]:
df_merged.head()

,Date,Ticker,Open,Low,Volume,High,Close
0,2015-07-01,TSLA,18.073999,17.856667,31518000,18.174667,17.943333
1,2015-07-02,TSLA,18.680000,18.220667,107458500,18.830000,18.667999
2,2015-07-06,TSLA,18.591999,18.420000,61828500,18.779333,18.648001
3,2015-07-07,TSLA,18.333332,17.384666,91576500,18.346666,17.858667
4,2015-07-08,TSLA,17.288000,16.954000,93316500,17.386667,16.997334
